In [1]:
import functools
from src.inference.vllm_inference import setup_model
from src.data.loader import load_benchmark
from src.evaluator import Evaluator
from src.metrics import PassAtk, GreedyPass, PercentPassed, MeanEntropy
from src.data.prompt import mbpp, humaneval
import datasets

In [2]:
llm, tokenizer, _ = setup_model(use_sft=False)

INFO 01-31 06:41:44 [utils.py:261] non-default args: {'trust_remote_code': True, 'seed': 42, 'max_model_len': 4096, 'enable_prefix_caching': False, 'gpu_memory_utilization': 0.7, 'disable_log_stats': True, 'max_lora_rank': None, 'model': './models/qwen3-0.6b'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 01-31 06:41:44 [model.py:541] Resolved architecture: Qwen3ForCausalLM
INFO 01-31 06:41:44 [model.py:1561] Using max model len 4096
INFO 01-31 06:41:44 [scheduler.py:226] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-31 06:41:44 [vllm.py:624] Asynchronous scheduling is enabled.
WARNING 01-31 06:41:44 [interface.py:470] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
WARNING 01-31 06:41:44 [system_utils.py:140] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: WSL is detected and NVML is not compatible with fork
(EngineCore_DP0 pid=6842) INFO 01-31 06:41:49 [core.py:96] Initializing a V1 LLM engine (v0.15.0) with config: model='./models/qwen3-0.6b', speculative_config=None, tokenizer='./models/qwen3-0.6b', skip_tokenizer_init=False, tokenizer_mode=auto, revis

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.73it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.73it/s]
(EngineCore_DP0 pid=6842) 


(EngineCore_DP0 pid=6842) INFO 01-31 06:41:52 [default_loader.py:291] Loading weights took 0.59 seconds
(EngineCore_DP0 pid=6842) INFO 01-31 06:41:52 [gpu_model_runner.py:4118] Model loading took 1.12 GiB memory and 1.103943 seconds
(EngineCore_DP0 pid=6842) INFO 01-31 06:41:56 [backends.py:805] Using cache directory: /root/.cache/vllm/torch_compile_cache/1722d68ac3/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=6842) INFO 01-31 06:41:56 [backends.py:865] Dynamo bytecode transform time: 4.09 s
(EngineCore_DP0 pid=6842) INFO 01-31 06:42:00 [backends.py:302] Cache the graph of compile range (1, 8192) for later use
(EngineCore_DP0 pid=6842) INFO 01-31 06:42:02 [backends.py:319] Compiling a graph for compile range (1, 8192) takes 1.72 s
(EngineCore_DP0 pid=6842) INFO 01-31 06:42:02 [monitor.py:34] torch.compile takes 5.81 s in total
(EngineCore_DP0 pid=6842) INFO 01-31 06:42:03 [gpu_worker.py:356] Available KV cache memory: 3.03 GiB
(EngineCore_DP0 pid=6842) INFO 01-31 06:4

(EngineCore_DP0 pid=6842) 2026-01-31 06:42:03,341 - INFO - autotuner.py:256 - flashinfer.jit: [Autotuner]: Autotuning process starts ...
(EngineCore_DP0 pid=6842) 2026-01-31 06:42:03,426 - INFO - autotuner.py:262 - flashinfer.jit: [Autotuner]: Autotuning process ends
Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:02<00:00, 17.86it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 19.85it/s]


(EngineCore_DP0 pid=6842) INFO 01-31 06:42:08 [gpu_model_runner.py:5051] Graph capturing finished in 5 secs, took -0.59 GiB
(EngineCore_DP0 pid=6842) INFO 01-31 06:42:08 [core.py:272] init engine (profile, create kv cache, warmup model) took 15.92 seconds
(EngineCore_DP0 pid=6842) INFO 01-31 06:42:09 [vllm.py:624] Asynchronous scheduling is enabled.
INFO 01-31 06:42:09 [llm.py:343] Supported tasks: ['generate']


In [3]:
mbpp_mapper = functools.partial(mbpp.mbpp_to_task, tokenizer=tokenizer)
he_mapper = functools.partial(humaneval.humaneval_to_task, tokenizer=tokenizer)

In [4]:
raw_mbpp = datasets.load_dataset("google-research-datasets/mbpp", "sanitized", split="test")
raw_he = datasets.load_dataset("openai/openai_humaneval", split="test")

In [5]:
tasks_mbpp = load_benchmark(raw_mbpp, mbpp_mapper)
tasks_he = load_benchmark(raw_he, he_mapper)

In [6]:
metrics = [
	GreedyPass(),
    PassAtk(k=1, n_samples=1),
    # PassAtk(k=10, n_samples=10),
	PercentPassed(),
    MeanEntropy(),
]

In [7]:
evaluator = Evaluator(llm, tokenizer, metrics)

In [8]:
print("=== MBPP RESULTS ===")
print(evaluator.run(tasks_mbpp))

=== MBPP RESULTS ===

🚀 Group: ['greedy@1']
⚙️ Params: n=1, temp=0.0, max_tokens=4096, logprobs=1


Adding requests:   0%|          | 0/257 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/257 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

🚀 Executing 257 samples in parallel using 8 workers...


Running Tests:   0%|          | 0/257 [00:00<?, ?it/s]

...

🚀 Executing 257 samples in parallel using 8 workers...


Running Tests:   0%|          | 0/257 [00:00<?, ?it/s]

[1, 2, 3, 4, 5][1, 2, 3, 4, 5]

[1, 2, 3, 4, 5]


<string>:2: SyntaxWarning: invalid escape sequence '\s'


📊 pass@1 (n=1): 0.4358
📊 mean_%passed: 0.4906
📊 mean_entropy: 0.3833
{'greedy@1': 0.45136186770428016, 'pass@1 (n=1)': np.float64(0.4357976653696498), 'mean_%passed': np.float64(0.4905966277561608), 'mean_entropy': np.float64(0.3833415454320567)}


In [ ]:
print("=== HUMANEVAL RESULTS ===")
print(evaluator.run(tasks_he))

=== HUMANEVAL RESULTS ===

🚀 Group: ['greedy@1']
⚙️ Params: n=1, temp=0.0, max_tokens=4096, logprobs=1


Adding requests:   0%|          | 0/164 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/164 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

🚀 Executing 164 samples in parallel using 8 workers...


Running Tests:   0%|          | 0/164 [00:00<?, ?it/s]

📊 greedy@1: 0.3598

🚀 Group: ['pass@1 (n=1)', 'mean_%passed', 'mean_entropy']
⚙️ Params: n=1, temp=0.6, max_tokens=4096, logprobs=1


Adding requests:   0%|          | 0/164 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/164 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

ERROR 01-31 07:03:33 [core_client.py:605] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.
